In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import math
import matplotlib.pyplot as plt
import h5py
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import load_model

In [ ]:
pre_trained = InceptionV3(input_shape = (100, 100, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in pre_trained.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(pre_trained.output)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.05)(x)
x = layers.Dense(500, activation = 'relu')(x)
x = layers.Dense(120, activation = 'softmax')(x)
model = Model(pre_trained.input, x)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
#hyper parameters
batch_size = 8
kernel_size = (2, 2)
filters = 64
epochs = 200
train_dir = #refer to data preprocessing script to set this path
test_dir =  #refer to data preprocessing script to set this path

In [ ]:
train_datagen = ImageDataGenerator(
                rescale = 1./255, 
                shear_range = 0.2, 
                horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_directory(train_dir, 
                                              batch_size = batch_size,
                                              target_size = (100, 100),
                                              color_mode = 'rgb',
                                              class_mode = 'categorical')

test_set = validate_datagen.flow_from_directory(test_dir, 
                                                batch_size = batch_size,
                                                target_size = (100, 100),
                                                color_mode = 'rgb',
                                                class_mode = 'categorical')

In [ ]:
#added features for accurate and optimized training
# a check point callback to save our best weights
filepath = #set path
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max', 
                             save_weights_only=False)

# a reducing lr callback to reduce lr when val_loss doesn't increase
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                   patience=1, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=2, min_lr=1e-7)

# for early stop
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience = 40)

In [ ]:
history = model.fit(train_set, 
          epochs = epochs, 
          validation_data = test_set,
          validation_steps = 50,
          verbose = 1, 
          steps_per_epoch = 1000, 
          callbacks=[checkpoint, reduce_lr, early_stop])